#### 关于torch.nn
- 使用pytorch来构建神经网络，主要工具都在torch.nn包中
- nn依赖于autograd来定义模型，并对其自动求导

#### 构建神经网络的典型流程

- 定义一个拥有可学习参数的神经网络
- 遍历训练数据集
- 处理输入数据使其流经神经网络
- 计算损失值
- 将网络参数的梯度进行反向传播
- 以一定的规则更新网络的权重

In [1]:
# 导入苦于工具包
import torch
import torch.nn as nn
import torch.nn.functional as F

# 定义一个简单的网络类
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 定义第一层的卷积神经网络，输入通道维度=1，输出通道维度=6.卷积核大小3*3
        self.conv1 = nn.Conv2d(1, 6, 3)
        # 定义第二层卷积神经网络，输入通道维度=6,输出通道维度=16，卷积核大小3*3
        self.conv2 = nn.Conv2d(6, 16, 3)
        # 定义第三层全连接网络
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 在(2, 2)的池化窗口下执行最大池化操作
        # 注意任意卷积层后面要加激活层，池化层
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # 经过卷积层处理后，张量要进入全连接层，进入前需要调整张量的形状
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        # 计算size, 除了第0个维度上的batch_size
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


#### 模型中所有的可训练参数，可以通过net.parameters()来获得

In [4]:
params = list(net.parameters())
print(params)
print(len(params))
print(params[0].size())

[Parameter containing:
tensor([[[[ 0.1816,  0.0147,  0.2244],
          [ 0.1292,  0.1517, -0.0884],
          [ 0.1117, -0.2432,  0.2255]]],


        [[[ 0.1592, -0.2725, -0.1455],
          [-0.2495, -0.2141,  0.2920],
          [ 0.0224, -0.0857, -0.1947]]],


        [[[ 0.1007,  0.2706, -0.2832],
          [ 0.1719,  0.0083,  0.3183],
          [-0.2251,  0.0013,  0.0948]]],


        [[[ 0.1859,  0.0681,  0.1555],
          [-0.1378,  0.0475, -0.1863],
          [-0.1889,  0.2218,  0.0465]]],


        [[[-0.1656, -0.0866, -0.0454],
          [ 0.0077, -0.1432,  0.3081],
          [-0.0744,  0.0683, -0.2291]]],


        [[[ 0.1331, -0.2354, -0.1769],
          [-0.2991, -0.0809, -0.2159],
          [-0.1794, -0.0724,  0.0489]]]], requires_grad=True), Parameter containing:
tensor([-0.3195,  0.2838, -0.2865,  0.3039, -0.0223,  0.0338],
       requires_grad=True), Parameter containing:
tensor([[[[-0.1089,  0.0135,  0.0281],
          [-0.0429,  0.0312,  0.1337],
          [-0.0328

#### 假设图像的输入尺寸为32*32

In [5]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.1022,  0.1310,  0.0664, -0.0728,  0.0986,  0.0410,  0.0835,  0.0882,
          0.0722, -0.0710]], grad_fn=<AddmmBackward0>)


#### 有了输出张量后，就可以执行梯度归零和反向传播的操作了

In [6]:
net.zero_grad()
out.backward(torch.randn(1, 10))

#### 注意
- torch.nn构建的神经网络只支持mini-batches的输入，不支持单一样本的输入
- 比如: nn.Conv2d需要一个4D的Tensor，形状为(nSamples, nChannels, Height, Width)如果你的输入只有单一样本形式，则只需要执行input.unsqueeze(0).主动将3D Tensor扩充成4D Tensor

#### 损失函数
- 损失函数的输入是一个输入的pair(output, target)然后计算出一个数值来评估output和target之间的差距大小
- 在torch.nn中在若干不同的损失函数可供使用，比如nn.MSELoss就是通过计算均方差损失来评估输入和目标值之间的差距

> 下面是一个MSTLoss计算损失的例子

In [10]:
output = net(input)
print(output)
target = torch.randn(10)
print(target)
# 改变target形状为二维张量，为了和output匹配
target = target.view(1, -1)
print(target)
criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)

tensor([[-0.1022,  0.1310,  0.0664, -0.0728,  0.0986,  0.0410,  0.0835,  0.0882,
          0.0722, -0.0710]], grad_fn=<AddmmBackward0>)
tensor([-1.6489,  0.2414, -2.5625, -0.2139,  0.2378,  1.3533, -1.4623, -0.0681,
         0.7402,  0.2568])
tensor([[-1.6489,  0.2414, -2.5625, -0.2139,  0.2378,  1.3533, -1.4623, -0.0681,
          0.7402,  0.2568]])
tensor(1.4045, grad_fn=<MseLossBackward0>)


- 关于方向传播的链条，如果我们跟踪loss反向传播方向，使用.grad_fn属性打印，将可以看到一张完整的计算图如下
- Input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d -> view -> linear -> relu -> linear -> relu -> linear -> MSELoss -> loss

- 当调用loss.backward()时，整张计算图将对loss进行自动求导，所有属性requires_grad=True的Tensors都将参与梯度求导的运算，并将梯度累加到Tensors中的.grad属性中

In [12]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

#### 反向传播(backpropagation)
- 在pytorch中执行反向传播非常方便，全部的操作就是loss.backward()
- 在执行反向传播前，要先将梯度清零，否则梯度会不同的批次数据之间被累加

In [13]:
# pytorch中执行梯度清零的代码
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

# pytorch中执行反向传播的代码
loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0123,  0.0403, -0.0084,  0.0084, -0.0032,  0.0161])


#### 更新网络参数
- 更新参数最简单的算法就是SGD(随机梯度下降)
- 具体的算法公式表达式为:weight = weight - learning_rate * gradient

> 首先用传统的python代码来实现SGD如下:

In [14]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

使用pytorch官方推荐的标准代码如下:

In [16]:
# 首先导入优化器的包，optim中包含若干常用的优化算法，比如SGD，Adam等
import torch.optim as optim

# 通过optim创建优化器对象
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 将优化器执行梯度清零操作
optimizer.zero_grad()

output = net(input)
loss = criterion(output, target)

# 对损失值执行反向传播的操作
loss.backward()
# 参数的更新通过一行标准代码来执行
optimizer.step()

#### 小节总结

- 学习了构建一个神经网络的典型流程
	-  定义一个拥有可学习参数的神经网络
	- 遍历训练数据集
	- 处理输入数据使其流经神经网络
	- 计算损失值
	- 将网络参数的梯度进行反向传播
	- 以一定的规则更新网络权重
- 学习了损失函数的定义
	- 采用torch.nn.MSELoss()计算均方误差
	- 通过loss.backward()进行反向传播计算时，整张计算图将对loss进行自动求导，所有属性requires_grad=True的Tensors都将参与梯度求导的运算，并将梯度累加到Tensors中的,grad属性中
- 学习了反向传播的计算方法
	- 在pytorch中执行反向传播非常简便，全部的操作就是loss.backward()
	- 在执行反向传播之前，要先将梯度清零，否则梯度会在不同的批次数据之间被累加
		- net.zero_grad()
		- loss.backward()
- 学习了参数更新方法
	- 定义优化器来执行参数的优化与更新
		- optimizer = optim.SGD(net.parameters(), lr=0.01)
	- 通过优化器来执行具体的参数更新
		- optimizer.step()
